In [1]:
import import_ipynb
from ResNet_Model import *
from fastai import *
from fastai.text import *
from nltk.tokenize import word_tokenize
from fastai.callback import *
import torch.utils.data as data_utils
import torch

importing Jupyter notebook from ResNet_Model.ipynb


ModuleNotFoundError: No module named 'fastai.vision.utils'

In [ ]:
device = torch.device('cuda')

In [ ]:
PADDING = 64  # 64 for Headlines, 128 for Reddit Main/Pol
DATASET_PATH = '' #Dataset folder path
DATASET = 'Headlines.csv'  #Dataset file
COL = 'headline'   #headline for Headlines, comment for Reddit Main/Pol

# Word embedding weights
WEIGHTS = 'Weights_glove_headlines.pkl'
EMBED_DIM = 50 # 50 for GloVe, 300 for FastText
STATIC = False #Static or non-static embeddings

In [ ]:
def pad_to(x: OrderedDict(), pad_til = PADDING) -> OrderedDict():
    res = []
    count = 0
    for t in x:
        res.append(t)
        count += 1
    while count < pad_til:
        res.append(PAD)
        count +=1
    return res

In [ ]:
tokenizer = Tokenizer(SpacyTokenizer, pre_rules=[fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces], post_rules=[replace_all_caps, deal_caps, pad_to])
processor = [TokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor()]

In [ ]:
data = (TextList.from_csv(DATASET_PATH, DATASET, cols=COL, processor=processor))

In [ ]:
data = data.split_from_df(col='valid').label_from_df(cols=0).databunch(num_workers=0)

In [ ]:
weights_matrix = pickle.load(open(WEIGHTS, 'rb'))

In [ ]:
net = ResNet(weights_matrix, embed_dim = EMBED_DIM, static = STATIC)

In [ ]:
net.to('cuda')

In [ ]:
learn = Learner(data, net, wd=0.1, loss_func=CrossEntropyFlat(),  metrics=[accuracy, FBeta(average='macro',beta=1),FBeta(average='micro',beta=1)])

In [ ]:
learn.fit_one_cycle(10, 1e-03, moms=(0.8,0.7))